# Comparación de Bases de Datos SQL y de Grafos (Neo4j) en Jupyter

Este tutorial te guiará a través de la comparación entre una base de datos relacional (SQL con SQLite) y una base de datos de grafos (Neo4j), enfocándonos en la carga de datos y la ejecución de consultas que involucran relaciones complejas. Utilizaremos un pequeño conjunto de datos de películas y actores para ilustrar los conceptos.

## 1. Preparación del Entorno

Para esta práctica, utilizaremos Docker para ejecutar Neo4j, lo que nos permite tener una base de datos funcional rápidamente. También necesitaremos Python y sus librerías.

### Instalar Docker

Docker es una plataforma que permite empaquetar aplicaciones en "contenedores". Neo4j se ejecutará dentro de un contenedor Docker.

* **Para Windows y macOS:**
    1.  Ve a la página oficial de Docker Desktop: [Get Docker Desktop](https://www.docker.com/products/docker-desktop)
    2.  Descarga e instala el software.
    3.  Abre Docker Desktop para asegurarte de que se inicie correctamente.

* **Para Linux:**
    1.  Sigue las instrucciones específicas para tu distribución Linux en la documentación oficial de Docker: [Install Docker Engine on Linux](https://docs.docker.com/engine/install/)
    2.  Verifica la instalación con `docker run hello-world`.

### Iniciar Neo4j con Docker

Una vez que Docker esté funcionando, puedes iniciar tu base de datos Neo4j con este comando. Ejecútalo en una **terminal separada** o en una celda de shell (con `!`) si tu entorno Jupyter lo permite.

In [ ]:
# Ejecuta este comando en una TERMINAL SEPARADA, no en esta celda de Jupyter si bloquea la ejecución.
# O usa '!' al inicio si tu entorno lo permite para ejecutar comandos de shell.

# !docker run \
#     --name neo4j-tutorial \
#     -p 7474:7474 -p 7687:7687 \
#     -e NEO4J_AUTH=neo4j/testpass \
#     neo4j:latest

Verifica que Neo4j esté corriendo accediendo a `http://localhost:7474` en tu navegador. Deberías poder iniciar sesión con `neo4j` / `testpass`.

### Instalar Python y Librerías

Asumimos que Python 3 y `pip` ya están instalados. Si no, instálalos desde [python.org](https://www.python.org/downloads/).

Ahora, instala la librería `neo4j` para Python. Si estás usando un entorno virtual (como el que crea Poetry), asegúrate de activarlo primero.

In [ ]:
!pip install neo4j

## 2. Creación del Dataset de Ejemplo

Usaremos un dataset muy pequeño directamente en nuestro código para evitar la gestión de archivos CSV. Esto es útil para demostraciones rápidas.

In [ ]:
# Datos de ejemplo de Películas y Personas
persons_data = [
    {'id': 'p1', 'name': 'Alice'},
    {'id': 'p2', 'name': 'Bob'},
    {'id': 'p3', 'name': 'Carol'},
    {'id': 'p4', 'name': 'David'}
]

movies_data = [
    {'id': 'm1', 'title': 'Movie A'},
    {'id': 'm2', 'title': 'Movie B'},
    {'id': 'm3', 'title': 'Movie C'}
]

relationships_data = [
    {'person_name': 'Alice', 'rel_type': 'ACTED_IN', 'movie_title': 'Movie A'},
    {'person_name': 'Bob', 'rel_type': 'ACTED_IN', 'movie_title': 'Movie A'},
    {'person_name': 'Bob', 'rel_type': 'ACTED_IN', 'movie_title': 'Movie B'},
    {'person_name': 'Carol', 'rel_type': 'ACTED_IN', 'movie_title': 'Movie B'},
    {'person_name': 'Alice', 'rel_type': 'DIRECTED', 'movie_title': 'Movie C'},
    {'person_name': 'David', 'rel_type': 'ACTED_IN', 'movie_title': 'Movie C'}
]

print("Dataset de ejemplo creado en memoria.")

## 3. Implementación y Consulta en SQLite (Base de Datos Relacional)

Usaremos SQLite, una base de datos relacional ligera que funciona sin necesidad de un servidor separado (la ejecutaremos en memoria).

In [ ]:
import sqlite3

print("\n--- SQL (SQLite) Example ---")

# 1. Crear una base de datos SQLite en memoria
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 2. Definir tablas
cursor.execute('''
CREATE TABLE Persons (
    id TEXT PRIMARY KEY,
    name TEXT
);
''')

cursor.execute('''
CREATE TABLE Movies (
    id TEXT PRIMARY KEY,
    title TEXT
);
''')

cursor.execute('''
CREATE TABLE Relationships (
    person_id TEXT,
    movie_id TEXT,
    rel_type TEXT,
    FOREIGN KEY (person_id) REFERENCES Persons(id),
    FOREIGN KEY (movie_id) REFERENCES Movies(id),
    PRIMARY KEY (person_id, movie_id, rel_type)
);
''')

# 3. Insertar datos
sql_persons = [(p['id'], p['name']) for p in persons_data]
sql_movies = [(m['id'], m['title']) for m in movies_data]
sql_relationships = []
for rel in relationships_data:
    person_id = next(p['id'] for p in persons_data if p['name'] == rel['person_name'])
    movie_id = next(m['id'] for m in movies_data if m['title'] == rel['movie_title'])
    sql_relationships.append((person_id, movie_id, rel['rel_type']))

cursor.executemany("INSERT INTO Persons VALUES (?,?)", sql_persons)
cursor.executemany("INSERT INTO Movies VALUES (?,?)", sql_movies)
cursor.executemany("INSERT INTO Relationships VALUES (?,?,?)", sql_relationships)
conn.commit()

# 4. Consultar: "Encuentra todos los actores que actuaron con Bob"
# Esto requiere múltiples JOINS para navegar las relaciones
query_sql = """
SELECT DISTINCT P2.name
FROM Persons AS P1
JOIN Relationships AS R1 ON P1.id = R1.person_id
JOIN Movies AS M ON R1.movie_id = M.id
JOIN Relationships AS R2 ON M.id = R2.movie_id
JOIN Persons AS P2 ON R2.person_id = P2.id
WHERE P1.name = 'Bob' AND R1.rel_type = 'ACTED_IN' AND R2.rel_type = 'ACTED_IN' AND P2.name != 'Bob';
"""

print("Executing SQL query: 'Find all actors who acted with Bob'")
cursor.execute(query_sql)
results_sql = cursor.fetchall()

print("Actors who acted with Bob (SQL):")
if results_sql:
    for row in results_sql:
        print(f"- {row[0]}")
else:
    print("No co-actors found for Bob.")

conn.close()

## 4. Implementación y Consulta en Neo4j (Base de Datos de Grafos)

Ahora, realizaremos la misma operación en Neo4j, utilizando el driver Python y el lenguaje de consulta Cypher. Asegúrate de que tu contenedor Neo4j Docker esté corriendo.

In [ ]:
from neo4j import GraphDatabase
import time

print("\n--- Neo4j (Graph Database) Example ---")

# 1. Conexión a la base de datos Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "testpass"

driver = None
try:
    driver = GraphDatabase.driver(uri, auth=(username, password))
    driver.verify_connectivity()
    print("Conexión a Neo4j exitosa.")
    time.sleep(1) # Pequeña pausa para asegurar la conexión
except Exception as e:
    print(f"Error al conectar a Neo4j: {e}")
    print("Asegúrate de que Neo4j Docker container esté corriendo (ver sección 1).")
    # Si no se puede conectar, salimos para evitar errores posteriores.
    raise e 

# Función auxiliar para ejecutar consultas Cypher
def execute_cypher(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return [record for record in result]

# 2. Limpiar la base de datos (para empezar de cero)
print("Limpiando la base de datos Neo4j...")
execute_cypher("MATCH (n) DETACH DELETE n")
print("Base de datos Neo4j limpia.")

# 3. Crear nodos de Personas y Películas
print("Creando nodos...")
for p in persons_data:
    execute_cypher("CREATE (p:Person {id: $id, name: $name})", p)
for m in movies_data:
    execute_cypher("CREATE (m:Movie {id: $id, title: $title})", m)
print("Nodos creados.")

# 4. Crear relaciones
print("Creando relaciones...")
for rel in relationships_data:
    query_rel = f"""
    MATCH (p:Person {{name: $person_name}}),
          (m:Movie {{title: $movie_title}})
    CREATE (p)-[:{rel['rel_type']}]->(m)
    """
    execute_cypher(query_rel, {'person_name': rel['person_name'], 'movie_title': rel['movie_title']})
print("Relaciones creadas en Neo4j.")

# 5. Consultar: "Encuentra todos los actores que actuaron con Bob"
# Esto utiliza la sintaxis de patrones de grafos para navegar relaciones
query_cypher = """
MATCH (bob:Person {name: 'Bob'})-[:ACTED_IN]->(movie:Movie)<-[:ACTED_IN]-(co_actor:Person)
WHERE co_actor.name <> 'Bob'
RETURN DISTINCT co_actor.name AS CoActor
"""

print("\nExecuting Cypher query: 'Find all actors who acted with Bob'")
results_cypher = execute_cypher(query_cypher)

print("Actors who acted with Bob (Cypher):")
if results_cypher:
    for record in results_cypher:
        print(f"- {record['CoActor']}")
else:
    print("No co-actors found for Bob.")

# 6. Cerrar el driver
if driver:
    driver.close()

## 5. Discusión y Comparación

Después de ejecutar ambos códigos, observa y compara los resultados y la sintaxis:

### Legibilidad y Complejidad de la Consulta

* **SQL (Relacional):** La consulta para "actores que actuaron con Bob" requiere múltiples `JOIN`s (`Persons` -> `Relationships` -> `Movies` -> `Relationships` -> `Persons`). A medida que las relaciones se vuelven más profundas o complejas, la consulta SQL puede volverse muy verbosa y difícil de leer y mantener.
    ```sql
    SELECT DISTINCT P2.name
    FROM Persons AS P1
    JOIN Relationships AS R1 ON P1.id = R1.person_id
    JOIN Movies AS M ON R1.movie_id = M.id
    JOIN Relationships AS R2 ON M.id = R2.movie_id
    JOIN Persons AS P2 ON R2.person_id = P2.id
    WHERE P1.name = 'Bob' AND R1.rel_type = 'ACTED_IN' AND R2.rel_type = 'ACTED_IN' AND P2.name != 'Bob';
    ```

* **Cypher (Grafos):** La misma consulta en Cypher es mucho más intuitiva y expresa directamente el patrón de relaciones:
    ```cypher
    MATCH (bob:Person {name: 'Bob'})-[:ACTED_IN]->(movie:Movie)<-[:ACTED_IN]-(co_actor:Person)
    WHERE co_actor.name <> 'Bob'
    RETURN DISTINCT co_actor.name AS CoActor
    ```
    El patrón `(nodo1)-[:RELACION]->(nodo2)` refleja de forma natural cómo las entidades están conectadas en el mundo real.

### Modelo de Datos

* **SQL:** Las relaciones se representan indirectamente a través de claves foráneas y tablas de unión. Para "navegar" de una entidad a otra, se requiere un `JOIN`.
* **Neo4j:** Las relaciones son entidades de primera clase, con sus propios tipos y dirección. Esto hace que el modelo sea más cercano al problema de dominio y facilita la navegación de las conexiones.

### Rendimiento (Conceptual)

* Para consultas que implican muchos "saltos" a través de relaciones (a menudo llamadas "multi-hop queries"), las bases de datos de grafos como Neo4j suelen ser más eficientes. Esto se debe a que las relaciones se almacenan como punteros directos entre nodos, lo que permite un recorrido muy rápido del grafo sin las costosas operaciones de `JOIN` de las bases de datos relacionales.

En resumen, mientras que SQL es excelente para datos tabulares y relaciones bien definidas, las bases de datos de grafos brillan cuando la **conectividad y las relaciones** son el aspecto más importante de tus datos, ofreciendo una mayor expresividad y eficiencia para consultas complejas en redes.